In [1]:
import pandas as pd
c=pd.read_csv('C:\\Fake news\\train.csv')
c.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
c=c.dropna()
c=c.reset_index()
c.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
X = c.drop(['index','label'],axis=1)
y = c['label']
X.shape

(18285, 4)

In [4]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [5]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [6]:
voc_size=5000

In [7]:
messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [8]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [11]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[637, 1748, 957, 4024, 781, 1343, 107, 2396, 3822, 3240],
 [930, 2951, 2826, 2119, 4397, 3213, 1855],
 [309, 1950, 4367, 3356],
 [1485, 3046, 3717, 446, 1940, 2284],
 [3692, 4397, 4318, 1048, 2553, 1075, 4397, 4209, 4533, 3767],
 [857,
  439,
  3495,
  3573,
  838,
  748,
  2717,
  4794,
  1431,
  2185,
  3285,
  2450,
  1008,
  4522,
  1855],
 [3081, 1336, 3237, 1928, 2785, 1552, 2838, 4474, 942, 1476, 1764],
 [1364, 2922, 571, 4332, 1816, 969, 748, 986, 942, 1476, 1764],
 [76, 4847, 3338, 3277, 1964, 2013, 347, 261, 748, 3948],
 [3426, 2464, 2894, 1824, 3776, 133, 933, 3749],
 [2433, 3295, 601, 984, 3616, 3842, 4624, 3249, 2875, 3293, 4794],
 [446, 3015, 781, 2013, 748, 1816],
 [3509, 3503, 3130, 4338, 2915, 3998, 3685, 1545, 4537],
 [4770, 783, 3358, 927, 617, 3666, 1073, 942, 1476, 1764],
 [4360, 4965, 4278, 4561, 1024, 942, 1476, 1764],
 [4061, 3913, 2141, 1247, 4671, 664, 4286, 360, 4919, 2386],
 [4575, 6, 2951],
 [3056, 4209, 4489, 1278, 748, 483, 4108, 1855],
 [4781, 3363, 282

In [12]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2396 3822 3240]
 [   0    0    0 ... 4397 3213 1855]
 [   0    0    0 ... 1950 4367 3356]
 ...
 [   0    0    0 ...  942 1476 1764]
 [   0    0    0 ...  847 1164  901]
 [   0    0    0 ... 3060 2291 4326]]


In [13]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [16]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 27s 36ms/step - loss: 0.3442 - accuracy: 0.8339 - val_loss: 0.1966 - val_accuracy: 0.9150
Epoch 2/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1350 - accuracy: 0.9461 - val_loss: 0.1970 - val_accuracy: 0.9193
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0891 - accuracy: 0.9668 - val_loss: 0.2172 - val_accuracy: 0.9152
Epoch 4/10
192/192 [==============================] - 5s 29ms/step - loss: 0.0579 - accuracy: 0.9793 - val_loss: 0.2998 - val_accuracy: 0.9162
Epoch 5/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0379 - accuracy: 0.9873 - val_loss: 0.3206 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0202 - accuracy: 0.9933 - val_loss: 0.4148 - val_accuracy: 0.9115
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0142 - accuracy: 0.9955 - val_loss: 0.5181 - val_accuracy: 0.911

In [17]:
y_pred=model.predict(X_test)
y_pred=[1 if x>=0.5 else 0 for x in y_pred]
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3096,  323],
       [ 193, 2423]], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9144987572493787

In [19]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 33ms/step - loss: 0.3461 - accuracy: 0.8330 - val_loss: 0.1989 - val_accuracy: 0.9178
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1452 - accuracy: 0.9413 - val_loss: 0.1966 - val_accuracy: 0.9167
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1069 - accuracy: 0.9596 - val_loss: 0.2130 - val_accuracy: 0.9196
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0788 - accuracy: 0.9707 - val_loss: 0.2429 - val_accuracy: 0.9157
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0556 - accuracy: 0.9808 - val_loss: 0.2662 - val_accuracy: 0.9125
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0403 - accuracy: 0.9863 - val_loss: 0.3452 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0319 - accuracy: 0.9900 - val_loss: 0.4042 - val_accuracy: 0.908

In [20]:
y_pred=model.predict(X_test)
y_pred=[1 if x>=0.5 else 0 for x in y_pred]
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3105,  314],
       [ 228, 2388]], dtype=int64)

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9101905550952776